In [2]:
import pm4py

In [3]:
log = pm4py.read_xes("3.xes")
log

d:\Programs\Programming\Python\lib\site-packages\pm4py\util\dt_parsing\parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]

,concept:name,ManagedThreadId,NativeThreadId,QpcStamp,case:concept:name,time:timestamp,concept:instance,lifecycle:transition
0,Procfiler/MethodStart_{Microsoft.CodeAnalysis....,25452,25452,13371926562902,1,NaT,NaN,NaN
1,GC/SampledObjectAllocation_{System.String},25452,25452,13371926565460,1,NaT,NaN,NaN
2,Procfiler/MethodStart_{Interop+Advapi32.EventA...,25452,25452,13371926565514,1,NaT,NaN,NaN
3,Procfiler/MethodEnd_{Interop+Advapi32.EventAct...,25452,25452,13371926565518,1,2025-01-10 20:01:47.094208+00:00,NaN,NaN
4,GC/SampledObjectAllocation_{ActivityInfo},25452,25452,13371926565530,1,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...
2341769,Method/LoadVerbose_{Microsoft.CodeAnalysis.Com...,-1,18968,13373031606221,12,NaT,MethodLoadUnload_Microsoft.CodeAnalysis.Comman...,start
2341770,Procfiler/MethodStart_{Microsoft.CodeAnalysis....,18968,18968,13373031606272,12,2025-01-10 20:03:37.599576+00:00,NaN,NaN
2341771,Procfiler/MethodEnd_{Microsoft.CodeAnalysis.Co...,18968,18968,13373031606318,12,NaT,NaN,NaN
2341772,Procfiler/MethodEnd_{Microsoft.CodeAnalysis.CS...,18968,18968,13373031606321,12,NaT,NaN,NaN


In [4]:
import re

# Функция для обрезки до 6 знаков после запятой
def trim_timestamp(timestamp):
    match = re.match(r"(.*\.\d{6})\d*(.*)", timestamp)
    if match:
        return match.group(1) + match.group(2)
    return timestamp

# Путь к файлам
input_file_path = './3.xes'
output_file_path = './3_modified.xes'

# Счетчик для атрибута value
trace_counter = 1
inside_trace = False

with open(input_file_path, 'r', encoding='utf-8') as infile, \
     open(output_file_path, 'w', encoding='utf-8') as outfile:
    
    for line in infile:
        if '<trace>' in line and not inside_trace:
            # Нашли начало нового trace, устанавливаем флаг
            inside_trace = True
            # Записываем строку <trace> и добавляем новый элемент <string>
            outfile.write(line)
            outfile.write(f'  <string key="concept:name" value="{trace_counter}"/>\n')
            trace_counter += 1
        elif '</trace>' in line and inside_trace:
            # Дошли до конца trace, сбрасываем флаг
            inside_trace = False
            outfile.write(line)
        elif 'key="time:timestamp"' in line:
            try:
                # Подстраиваем timestamp до 6 знаков после точки
                timestamp = re.search('value="([^"]*)"', line).group(1)
                trimmed_value = trim_timestamp(timestamp)
                modified_line = re.sub('value="([^"]*)"', f'value="{trimmed_value}"', line)
                outfile.write(modified_line)
            except Exception as e:
                print(f"Ошибка при обработке строки с timestamp: {line.strip()}\nОшибка: {e}")
                outfile.write(line)
        else:
            # Для всех остальных строк
            outfile.write(line)

In [5]:
log = pm4py.read_xes("3_modified.xes")
log

parsing log, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]

,concept:name,time:timestamp,ManagedThreadId,NativeThreadId,QpcStamp,case:concept:name,concept:instance,lifecycle:transition
0,Procfiler/MethodStart_{Microsoft.CodeAnalysis....,2025-01-10 20:01:47.093946+00:00,25452,25452,13371926562902,1,NaN,NaN
1,GC/SampledObjectAllocation_{System.String},2025-01-10 20:01:47.094202+00:00,25452,25452,13371926565460,1,NaN,NaN
2,Procfiler/MethodStart_{Interop+Advapi32.EventA...,2025-01-10 20:01:47.094207+00:00,25452,25452,13371926565514,1,NaN,NaN
3,Procfiler/MethodEnd_{Interop+Advapi32.EventAct...,2025-01-10 20:01:47.094208+00:00,25452,25452,13371926565518,1,NaN,NaN
4,GC/SampledObjectAllocation_{ActivityInfo},2025-01-10 20:01:47.094209+00:00,25452,25452,13371926565530,1,NaN,NaN
...,...,...,...,...,...,...,...,...
2341769,Method/LoadVerbose_{Microsoft.CodeAnalysis.Com...,2025-01-10 20:03:37.599570+00:00,-1,18968,13373031606221,12,MethodLoadUnload_Microsoft.CodeAnalysis.Comman...,start
2341770,Procfiler/MethodStart_{Microsoft.CodeAnalysis....,2025-01-10 20:03:37.599576+00:00,18968,18968,13373031606272,12,NaN,NaN
2341771,Procfiler/MethodEnd_{Microsoft.CodeAnalysis.Co...,2025-01-10 20:03:37.599580+00:00,18968,18968,13373031606318,12,NaN,NaN
2341772,Procfiler/MethodEnd_{Microsoft.CodeAnalysis.CS...,2025-01-10 20:03:37.599580+00:00,18968,18968,13373031606321,12,NaN,NaN
